In [7]:
!pip install -q datasets transformers accelerate

In [16]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [9]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00


In [13]:
!pip install -U transformers datasets evaluate

In [15]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate

# Конфигурация
task = "sst2"  # Stanford Sentiment Treebank
model_checkpoint = "distilbert-base-uncased"
batch_size = 32
num_train_epochs = 1

# Загрузка метрики
accuracy_metric = evaluate.load("accuracy")

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [ ]:
try:
    # Стандартный способ загрузки GLUE SST-2
    dataset = load_dataset("glue", task)
    print("Данные успешно загружены через glue")
except Exception as e:
    print(f"Ошибка при загрузке через glue: {e}")
    # Альтернативный способ загрузки
    print("Попытка загрузки через sst2 напрямую...")
    dataset = load_dataset("sst2")
    print("Данные успешно загружены через sst2")

# Проверка структуры данных
print("\nКлючи датасета:", list(dataset.keys()))
print("Структура тренировочного набора:", dataset["train"])

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Данные успешно загружены через glue

Ключи датасета: ['train', 'validation', 'test']
Структура тренировочного набора: Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})


In [ ]:
# Определение ключей в зависимости от структуры датасета
if 'sentence' in dataset["train"].features:
    text_key = 'sentence'
elif 'text' in dataset["train"].features:
    text_key = 'text'
else:
    # Если ни один ключ не найден, используем первый подходящий
    text_key = [k for k in dataset["train"].features.keys() if k != 'label'][0]
    print(f"Используем ключ '{text_key}' для текста")

print(f"Используемый ключ для текста: {text_key}")

# Инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# Функция предобработки
def preprocess_function(examples):
    return tokenizer(
        examples[text_key],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

# Применение токенизации
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Проверка
print("\nПример предобработанных данных:")
print(encoded_dataset["train"][0])

Используемый ключ для текста: sentence


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]


Пример предобработанных данных:
{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2  # 2 класса для бинарной классификации
)

print(f"\nМодель {model_checkpoint} успешно загружена!")
print(f"Количество параметров: {model.num_parameters():,}")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Модель distilbert-base-uncased успешно загружена!
Количество параметров: 66,955,010


In [ ]:
# Функция вычисления метрик
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# ИСПРАВЛЕННЫЕ параметры обучения
training_args = TrainingArguments(
    output_dir=f"{model_checkpoint}-finetuned-{task}",
    eval_strategy="epoch",  # Исправлено с evaluation_strategy на eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-6-3253567381>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("Начало обучения модели...")
train_results = trainer.train()
print("\nОбучение завершено!")
trainer.save_model("best_model")
print("Лучшая модель сохранена в 'best_model'")

Начало обучения модели...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.171000,0.276721,0.903670


Epoch,Training Loss,Validation Loss,Accuracy
1,0.171000,0.276721,0.903670
2,0.109300,0.332667,0.893349
3,0.091900,0.394394,0.897936



Обучение завершено!
Лучшая модель сохранена в 'best_model'


In [ ]:
# Оценка модели
def safe_evaluate(dataset_name):
    """Безопасная оценка модели с обработкой ошибок"""
    try:
        # Проверка наличия набора данных
        if dataset_name not in encoded_dataset:
            print(f"Набор {dataset_name} не найден в датасете")
            return None

        # Проверка и исправление меток
        labels = encoded_dataset[dataset_name]["labels"]

        # Убедимся, что метки в правильном формате
        if any(not isinstance(label, int) for label in labels):
            print(f"Обнаружены нечисловые метки в {dataset_name}")
            return None

        # Проверка диапазона меток
        min_label = min(labels)
        max_label = max(labels)
        if min_label < 0 or max_label > 1:
            print(f"Метки в {dataset_name} вне диапазона [0, 1]: min={min_label}, max={max_label}")
            print("Исправление меток...")
            fixed_labels = [min(max(label, 0), 1) for label in labels]
            encoded_dataset[dataset_name] = encoded_dataset[dataset_name].remove_columns(["labels"])
            encoded_dataset[dataset_name] = encoded_dataset[dataset_name].add_column("labels", fixed_labels)

        # Выполнение оценки
        return trainer.evaluate(encoded_dataset[dataset_name])
    except Exception as e:
        print(f"Ошибка при оценке на {dataset_name}: {e}")
        return None

# Оценка на тестовом наборе (если доступен)
test_results = safe_evaluate("test")
if test_results:
    print("\nРезультаты на тестовом наборе:")
    print(f"Точность: {test_results['eval_accuracy']:.4f}")
    print(f"Потери: {test_results['eval_loss']:.4f}")
else:
    print("\nТестирование на тестовом наборе пропущено")

# Оценка на валидационном наборе
eval_results = safe_evaluate("validation")
if eval_results:
    print("\nРезультаты на валидационном наборе:")
    print(f"Точность: {eval_results['eval_accuracy']:.4f}")
    print(f"Потери: {eval_results['eval_loss']:.4f}")
else:
    print("\nОценка на валидационном наборе не удалась")

Ошибка при оценке на test: "Column labels not in the dataset. Current columns in the dataset: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask']"

Тестирование на тестовом наборе пропущено
Ошибка при оценке на validation: "Column labels not in the dataset. Current columns in the dataset: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask']"

Оценка на валидационном наборе не удалась


In [ ]:
# Инференс модели

from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="best_model",
    tokenizer=tokenizer
)

test_texts = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "I hated this film. It was the worst experience of my life.",
    "The acting was mediocre but the plot was engaging enough to keep me watching.",
    "An outstanding performance by the lead actor, though the script could have been better.",
    "Not terrible, but certainly not worth the price of admission."
]

print("\nТестовые предсказания:")
for i, text in enumerate(test_texts):
    result = classifier(text)[0]
    sentiment = "POSITIVE" if result['label'] == 'LABEL_1' else "NEGATIVE"
    print(f"\nПример {i+1}:")
    print(f"Текст: {text}")
    print(f"  Предсказание: {sentiment}")
    print(f"  Уверенность: {result['score']:.4f}")

Device set to use cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
!optimum-cli export onnx --help

2025-06-16 13:20:12.400860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750080012.420142   42494 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750080012.427108   42494 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: optimum-cli export onnx [-h] -m MODEL [--task TASK] [--opset OPSET]
                               [--device DEVICE] [--fp16]
                               [--dtype {fp32,fp16,bf16}]
                               [--optimize {O1,O2,O3,O4}] [--monolith]
                               [--no-post-process] [--variant VARIANT]
                               [--framework {pt,tf}] [--atol ATOL]
                               [--cac

In [6]:
!pip install optimum

In [ ]:
!optimum-cli export onnx --model best_model --task text-classification best_model_onnx/

/bin/bash: line 1: optimum-cli: command not found


In [ ]:
!pip install -U optimum[exporters] onnx onnxruntime onnxruntime-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


In [1]:
import onnx

model = onnx.load(f"{output_dir}/model.onnx")
onnx.checker.check_model(model)

print("valid")

ModuleNotFoundError: No module named 'onnx'